In [ ]:
from google.cloud import bigquery
import os, csv, xmltodict, json

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../creds.json"

In [ ]:
PROJECT = 'eng-reactor-287421'
dataset = 'zz_testing'
table = 'ice_nesting_test'

### make schema from ICE dictionary csv

In [ ]:
file_dir = 'C:/Users/vcrom/Desktop/ficc/Files/'

In [ ]:
with open(file_dir+'allcats.csv') as all_cats:
    read_file = csv.reader(all_cats)
    paths = list(read_file)
    

In [ ]:
split_paths = [[entry[0].split('/')[1:],[entry[1],entry[2]]] for entry in paths]

In [ ]:
repeated_leafs = [entry[0] for entry in paths if entry[2]=='REPEATED']

In [ ]:
run_dict = {}
for entry in split_paths[1:]:
    entry_path, entry_details = entry
    current_dict_level = run_dict
    for current_index in range(len(entry_path)-1):
        if entry_path[current_index] not in current_dict_level:
            current_dict_level[entry_path[current_index]] = {}
        current_dict_level = current_dict_level[entry_path[current_index]]

    try:
        current_dict_level[entry_path[-1]]=entry_details
    except:
        print('/'.join(entry[0]))
        
        
        

In [ ]:
repeater_test = ['/instrument/debt/coupon_payment_feature/coupon_date_rules/holiday_calendar','/instrument/debt/coupon_payment_feature/pik_details/step_schedule/rate','/instrument/debt/event','/instrument/debt/floating_rate_instruments/FRI_coupon_reset/cap_amount','/instrument/debt/floating_rate_instruments/FRI_coupon_reset/floor_amount','/instrument/debt/floating_rate_instruments/FRI_coupon_reset/max_rate','/instrument/debt/floating_rate_instruments/FRI_coupon_reset/min_rate','/instrument/debt/mbs_statistics/monthly/CPR','/instrument/debt/mbs_statistics/monthly/PSA','/instrument/debt/step_schedule/rate','/instrument/derivatives/options/contract/series/expiry/calls/strike','/instrument/derivatives/options/contract/series/expiry/deliverable/underlying/market/xref','/instrument/derivatives/options/contract/series/expiry/puts/strike','/instrument/derivatives/strategies/contract/series/market/deliverable/underlying/market/xref','/instrument/derivatives/strategies/contract/series/market/leg_details/leg/underlying/instrument_xref/xref','/instrument/derivatives/strategies/contract/series/market/leg_details/leg/underlying/market/xref','/instrument/global_information/organization_information/classifications/ICB2','/instrument/global_information/ratings/rating','/instrument/global_information/relations/instrument_roles/role','/instrument/master_information/instrument_xref/xref','/instrument/master_information/market_master/market/linked_markets/linked_market/xref','/instrument/master_information/market_master/market/xref','/instrument/master_information/organization_master/organization_xref/xref','/instrument/debt/sink_schedule','/instrument/debt/call_details/call_schedule','/instrument/debt/muni_details/linkage_child/linkage','/instrument/debt/muni_details/linkage_child/linkage/instrument_xref/xref','/instrument/debt/muni_escrow','/instrument/global_information/instrument_details/default_status','/instrument/debt/amount_outstanding_history/change','/instrument/debt/partial_redemption','/instrument/debt/default_event','/instrument/debt/floating_rate_history','/instrument/global_information/instrument_details/conversion_details','/instrument/debt/loc_details','/instrument/debt/additional_sink_schedule','/instrument/global_information/organization_information/classifications/NAICS',
                    ]

In [ ]:
def bq_schema_maker(tree_dict,repeaters,path_string=''):
    schema_list = []
    for key in tree_dict.keys():
        if not isinstance(tree_dict[key],dict):
            bq_type, repeated = tree_dict[key]
            schema_list.append(bigquery.SchemaField(key,bq_type,mode=repeated))
        else:
            #how to get nullable vs repeated for top categories?
            new_path_string = path_string+'/'+key
            repeated_mode = "REPEATED" if new_path_string in repeaters else "NULLABLE"
            schema_list.append(bigquery.SchemaField(key,"RECORD",mode=repeated_mode,fields=bq_schema_maker(tree_dict[key],repeaters,new_path_string)))
    return schema_list

In [ ]:
test_ice_schema = bq_schema_maker(run_dict,repeater_test)

### BigQuery API

In [ ]:
def add_to_bigquery(rows_to_insert, table_id):
    bq_client = bigquery.Client()
    error = bq_client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    return error

### push schema to BigQuery

In [ ]:
bqclient = bigquery.Client(project=PROJECT,)

In [ ]:
table_id = '{}.{}.{}'.format(PROJECT,dataset,table)

In [ ]:
# delete table
#bqclient.delete_table(table_id, not_found_ok=True)

In [ ]:
# create a new table

test_ice_table = bigquery.Table(table_id, schema=test_ice_schema)
test_ice_table = bqclient.create_table(test_ice_table)

In [ ]:
# update schema of existing table
test_ice_table = bqclient.get_table(table_id)
test_ice_table.schema = test_ice_schema
test_ice_table = bqclient.update_table(test_ice_table, ["schema"])


In [ ]:
# get table
test_ice_table = bqclient.get_table(table_id)

In [ ]:
test_ice_table.schema[0].fields[1].fields[0].fields[0]

In [ ]:
#instrument.debt.put_details.put_type
test_ice_table.schema[0].fields[1].fields[1]

In [ ]:
bqclient.insert_rows_json(table_id,test_a_json)

### testing loading script

In [ ]:
xml_str = get_next_instrument_xml()

In [ ]:
instrument_dict = xmltodict.parse(xml_str,attr_prefix='',cdata_key='text')

In [ ]:
repeated_enforcement(test_address_list,instrument_dict)

In [ ]:
read_json_test = json.dumps(instrument_dict,indent=3)

In [ ]:
print(read_json_test)

In [ ]:
test_a_json = [json.loads(json.dumps(instrument_dict))]

In [ ]:
for hundreds in range(1000):
    test_a_json = []
    for i in range(100):
        xml_str = get_next_instrument_xml()
        instrument_dict = xmltodict.parse(xml_str,attr_prefix='',cdata_key='text')
        repeated_enforcement(test_address_list,instrument_dict)
        record_key_flatten(test_address_key_list,instrument_dict)
        derepeat_bad_fields(test_bad_address_list,instrument_dict)
        test_a_json.append(json.loads(json.dumps(instrument_dict)))

    #error = bqclient.insert_rows_json(table_id,test_a_json)
    error = add_to_bigquery(test_a_json,table_id)
    if len(error)>0: 
        if error[0]['errors'][0]['message'].startswith('Repeated record with'):
            print('rerun repeated_enforcement')
            error_index = error[0]['index']
            bad_enforcement = test_a_json[error_index]
            repeated_enforcement(test_address_list,bad_enforcement)
            test_a_json[error_index]
            error = add_to_bigquery(test_a_json,table_id)
            
            if len(error)>0:
                print(error)
                break
            else:
                print((hundreds+1)*100)
        else:
            print(error)
            break
    else:
        print((hundreds+1)*100)
        #print('loaded')
        


In [ ]:
test_a_json[31]['instrument']['master_information']['market_master']['market']['xref']

In [ ]:
test_a_json = [test_a_json[54]]

In [ ]:
test_a_json[error[0]['index']]

In [ ]:
test_a_json[0]

### Read one record at a time

In [ ]:
save_path = 'C:/Users/vcrom/Desktop/ficc/Files/'
file_name = 'gsm_init_muni_APFICC_GSMF10I.1.1_1.20201222T1700-05.xml'

f = open(save_path+file_name,'r')

In [ ]:
def get_next_instrument_xml():
    instrument_xml = ''
    while instrument_xml[-13:]!='</instrument>':
        instrument_xml += f.read(1)
    if instrument_xml.find('<payload>') != -1: instrument_xml = instrument_xml.split('<payload>')[1]
        
    return(instrument_xml)

In [ ]:
f.close()

### Modification functions

In [ ]:
test_address_list = repeater_test + repeated_leafs

In [ ]:
def repeated_enforcement(address_list,xml_dict):
    split_address_list = [entry.split('/')[1:] for entry in address_list]
    
    for address in split_address_list:
        current_xml_level = xml_dict
        for current_index in range(len(address)-1):
            if address[current_index] not in current_xml_level:
                break
            current_xml_level = current_xml_level[address[current_index]]

        if address[-1] not in current_xml_level:
            continue
        else:
            if not isinstance(current_xml_level[address[-1]],list):
                #print("repeating")
                current_xml_level[address[-1]]=[current_xml_level[address[-1]]]


In [ ]:
test_a_json = test_a_json[0]
repeated_enforcement(test_address_list,test_a_json)
test_a_json = [test_a_json]

In [ ]:
test_address_key_list = [['/instrument/debt/put_details/put_type','text'],
                        ['/instrument/master_information/market_master/market/linked_markets/linked_market/effective_date','text'],
                        ['/instrument/debt/floating_rate_instruments/benchmark_code','text'],
                        ['/instrument/debt/additional_sink_details/sink_amount_type','text'],
                        ['/instrument/master_information/market_master/market/linked_markets/linked_market/effective_date','text']]

In [ ]:
def record_key_flatten(address_key_list,xml_dict):
    split_address_key_list = [[entry[0].split('/')[1:],entry[1]] for entry in address_key_list]
    
    for entry in split_address_key_list:
        address, key = entry
        current_xml_level = xml_dict
        for current_index in range(len(address)-1):
            if address[current_index] not in current_xml_level:
                break
            current_xml_level = current_xml_level[address[current_index]]
            
        if address[-1] not in current_xml_level:
            continue
        else:
            if isinstance(current_xml_level[address[-1]],dict):
                #print('fixing')
                current_xml_level[address[-1]]= current_xml_level[address[-1]][key]

        

In [ ]:
test_a_json = test_a_json[0]
record_key_flatten(test_address_key_list,test_a_json)
test_a_json = [test_a_json]

In [ ]:
test_bad_address_list = ['/instrument/master_information/market_master/market']

In [ ]:
def derepeat_bad_fields(bad_address_list,xml_dict):
    split_bad_address_list = [entry.split('/')[1:] for entry in bad_address_list]
    
    for address in split_bad_address_list:
        current_xml_level = xml_dict
        for current_index in range(len(address)-1):
            if address[current_index] not in current_xml_level:
                break
            current_xml_level = current_xml_level[address[current_index]]
            
        if address[-1] not in current_xml_level:
            continue
        else:
            if isinstance(current_xml_level[address[-1]],list):
                print('derepeating')
                current_xml_level[address[-1]]= current_xml_level[address[-1]][0]

        

In [ ]:
test_a_json = test_a_json[0]
derepeat_bad_fields(test_bad_address_list,test_a_json)
test_a_json = [test_a_json]

### Download XML file

In [ ]:
from google.cloud import storage
client = storage.Client()
bucket = client.get_bucket('ref_data_1')

save_path = 'C:/Users/vcrom/Desktop/ficc/Files/'
file_name = 'gsm_init_muni_APFICC_GSMF10I.1.1_1.20201222T1700-05.xml'

blob = bucket.blob(file_name)

blob.download_to_filename(save_path+file_name)

